In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tiktok/tiktok_dataset.csv


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

df = pd.read_csv('/kaggle/input/tiktok/tiktok_dataset.csv')
df = df.dropna(subset=['video_view_count', 'video_transcription_text'])

df['viral'] = (df['video_view_count'] >= 100000).astype(int)

df = df.sample(n=1000, random_state=42)

df['video_transcription_text'] = df['video_transcription_text'].str.slice(0, 500)

X = df[['video_duration_sec', 'video_transcription_text', 
        'claim_status', 'verified_status', 'author_ban_status']]
y = df['viral']

text = ('caption', TfidfVectorizer(max_features=300, stop_words='english'), 'video_transcription_text')
cat = ('categoricals', OneHotEncoder(handle_unknown='ignore'), ['claim_status', 'verified_status', 'author_ban_status'])
num = ('numerics', 'passthrough', ['video_duration_sec'])

preprocessor = ColumnTransformer([text, cat, num])
model = make_pipeline(preprocessor, RandomForestClassifier(random_state=42))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
